In [1]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# https://github.com/rasbt/watermark
!pip install -q -U watermark

In [1]:
# importa várias bibliotecas do PyTorch necessárias para criar e treinar redes neurais
import torch  # Importa o módulo principal do PyTorch
import torchvision  # Importa o módulo torchvision para trabalhar com conjuntos de dados populares e modelos de redes neurais pré-treinados
import torchvision.transforms as transforms  # Importa o módulo torchvision.transforms para realizar transformações nos dados
import torch.nn as nn  # Importa o módulo nn do PyTorch para criar redes neurais
import torch.nn.functional as F  # Importa o módulo functional do PyTorch para utilizar funções de ativação e outras operações
import torch.optim as optim  # Importa o módulo optim do PyTorch para otimização de modelos de redes neurais
from torchsummary import summary  # Importa a função summary para apresentar o resumo do modelo

import time
import pandas as pd

In [2]:
# Carrega a extensão `watermark` para exibir informações sobre a versão do Python e dos pacotes instalados.
%reload_ext watermark

# Exibe as versões do Python e dos pacotes instalados.
%watermark --python --iversions

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.11.0

torch      : 2.0.0
pandas     : 1.5.3
torchvision: 0.15.1



In [3]:
# verifica se a biblioteca CUDA do PyTorch está disponível no sistema.
# Se estiver disponível, o dispositivo é definido como "cuda", indicando que a GPU será utilizada.
# Caso contrário, o dispositivo é definido como "cpu", indicando que a CPU será utilizada para a computação.
# O dispositivo escolhido será usado para alocar e executar os tensores do PyTorch.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exibe o dispositivo selecionado
device

device(type='cuda')

In [4]:
# Classe Net que define a arquitetura de uma rede neural convolucional (CNN)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Definição das camadas da rede neural convolucional
        self.conv1 = nn.Conv2d(1, 6, 5)  # Primeira camada de convolução: entrada com 1 canal, 6 filtros de tamanho 5x5
        self.pool = nn.MaxPool2d(2, 2)  # Camada de pooling: realiza o downsampling com um filtro de tamanho 2x2 e stride 2
        self.conv2 = nn.Conv2d(6, 16, 5)  # Segunda camada de convolução: entrada com 6 canais, 16 filtros de tamanho 5x5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # Primeira camada fully connected (FC): 16 * 4 * 4 entradas, 120 saídas
        self.fc2 = nn.Linear(120, 84)  # Segunda camada FC: 120 entradas, 84 saídas
        self.fc3 = nn.Linear(84, 10)  # Terceira camada FC: 84 entradas, 10 saídas (número de classes)

    def forward(self, x):
        # Propagação dos dados através das camadas da rede
        x = self.pool(F.relu(self.conv1(x)))  # Aplica a primeira camada de convolução, seguida de uma função de ativação ReLU e pooling
        x = self.pool(F.relu(self.conv2(x)))  # Aplica a segunda camada de convolução, seguida de uma função de ativação ReLU e pooling
        x = x.view(-1, 16 * 4 * 4)  # Redimensiona o tensor para ser compatível com a camada fully connected
        x = F.relu(self.fc1(x))  # Aplica a primeira camada fully connected, seguida de uma função de ativação ReLU
        x = F.relu(self.fc2(x))  # Aplica a segunda camada fully connected, seguida de uma função de ativação ReLU
        x = self.fc3(x)  # Aplica a terceira camada fully connected
        return x

In [5]:
# O método responsável por carregar os conjuntos de dados de treinamento e teste 
def load_dataset(dataset='MNIST', path='./data', batch_size=32, num_workers=1):
    # Define a transformação a ser aplicada aos dados
    transform = transforms.Compose(
    [transforms.ToTensor(),  # Converte as imagens em tensores
     transforms.Normalize((0.5,), (0.5,))])  # Normaliza as imagens com média 0.5 e desvio padrão 0.5
    
    if dataset == 'MNIST':
        # Carrega o conjunto de dados de treinamento do MNIST
        trainset = torchvision.datasets.MNIST(root=path, train=True,
                                              download=True, transform=transform)
        
        # Carrega o conjunto de dados de teste do MNIST
        testset = torchvision.datasets.MNIST(root=path, train=False,
                                             download=True, transform=transform)
    elif dataset == 'FashionMNIST':
        # Carrega o conjunto de dados de treinamento do FashionMNIST
        trainset = torchvision.datasets.FashionMNIST(root=path, train=True,
                                                     download=True, transform=transform)
        
        # Carrega o conjunto de dados de teste do FashionMNIST
        testset = torchvision.datasets.FashionMNIST(root=path, train=False,
                                                    download=True, transform=transform)
    else:
        raise ValueError('Dataset not supported. Please choose either "MNIST" or "FashionMNIST".')
    
    # Cria um dataloader para o conjunto de dados de treinamento
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)

    # Cria um dataloader para o conjunto de dados de teste
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)
    
    # Retorna os dataloaders do conjunto de treinamento e teste
    return trainloader, testloader


In [25]:
training_time_list = []
testing_time_list = []
accuracy_test_list = []
loss_test_list = []
epochs = range(2)

In [26]:
net = Net().to(device)  # Instancia o modelo e o move para a GPU, se disponível

criterion = nn.CrossEntropyLoss().to(device)  # Mova a função de perda para a GPU, se disponível
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Define o otimizador para atualizar os parâmetros do modelo

In [27]:
print('[torch-summary] Model Summary with torchvision.datasets.MNIST')
summary(net, (1, 28, 28))  # Resume o modelo, fornecendo o tamanho de entrada (1 canal, 28x28 pixels)

[torch-summary] Model Summary with torchvision.datasets.MNIST
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 120]          30,840
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [28]:
trainloader, testloader = load_dataset()  # Carrega os conjuntos de dados de treinamento e teste

loss_list = []

print('Starting training with %d images' % ( len(trainloader.dataset)))

start_time = time.time()
for epoch in epochs:
    running_loss = 0.0  # Variável para armazenar a perda acumulada durante o treinamento
    
   
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Mova os dados para a GPU, se disponível

        optimizer.zero_grad()  # Zera os gradientes dos parâmetros

        outputs = net(inputs)  # Propagação direta dos dados através do modelo
        loss = criterion(outputs, labels)  # Calcula a perda

        loss.backward()  # Retropropagação para calcular os gradientes
        optimizer.step()  # Atualiza os parâmetros do modelo com base nos gradientes

        running_loss += loss.item()  # Acumula a perda para fins de exibição
            
        if i == len(trainloader) - 1:
            last_loss = running_loss /  ((i % 100)+1)
            print('[%d, %5d] Last loss: %.3f' % (epoch + 1, i+1, last_loss  ) )
            loss_list.append(last_loss )  # Armazena o loss médio da época na lista

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0       

end_time = time.time()
duration = end_time - start_time
training_time_list.append(duration)
# Converte a duração para o formato hh:mm:ss
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
seconds = int(duration % 60)

print('Finished Training')
print(f"\nTraining Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
print('Training Time Duration: ',duration)


correct = 0  # Variável para armazenar o número de previsões corretas
total = 0  # Variável para armazenar o número total de exemplos de teste

start_time = time.time()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Mova os dados para a GPU, se disponível

        outputs = net(images)  # Propagação direta dos dados de teste através do modelo
        _, predicted = torch.max(outputs.data, 1)  # Obtém as previsões com maior probabilidade

        total += labels.size(0)  # Atualiza o número total de exemplos de teste
        correct += (predicted == labels).sum().item()  # Conta o número de previsões corretas

end_time = time.time()
duration = end_time - start_time
testing_time_list.append(duration)        
        
accuracy_test = correct / total
loss_test = (total - correct) / total
        
accuracy_test_pct = 100 * accuracy_test
loss_test_pct = 100 * loss_test

accuracy_test_list.append(accuracy_test)
loss_test_list.append(loss_test)

# Converte a duração para o formato hh:mm:ss
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
seconds = int(duration % 60)
print(f"\nTesting Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
print('Testing Time Duration: ',duration)

print('\nAccuracy of the network on the %d test images: %.3f' % (len(testloader.dataset), accuracy_test))
print('Loss of the network on the %d test images: %.3f' % (len(testloader.dataset), loss_test))
print('\nAccuracy of the network on the %d test images: %.3f %%' % (len(testloader.dataset), accuracy_test_pct))
print('Loss of the network on the %d test images: %.3f %%' % (total, loss_test_pct))

Starting training with 60000 images
[1,   100] loss: 1.325
[1,   200] loss: 0.412
[1,   300] loss: 0.264
[1,   400] loss: 0.221
[1,   500] loss: 0.179
[1,   600] loss: 0.154
[1,   700] loss: 0.143
[1,   800] loss: 0.134
[1,   900] loss: 0.103
[1,  1000] loss: 0.120
[1,  1100] loss: 0.103
[1,  1200] loss: 0.109
[1,  1300] loss: 0.102
[1,  1400] loss: 0.104
[1,  1500] loss: 0.097
[1,  1600] loss: 0.076
[1,  1700] loss: 0.076
[1,  1800] loss: 0.066
[1,  1875] Last loss: 0.081
[2,   100] loss: 0.072
[2,   200] loss: 0.080
[2,   300] loss: 0.062
[2,   400] loss: 0.072
[2,   500] loss: 0.055
[2,   600] loss: 0.073
[2,   700] loss: 0.061
[2,   800] loss: 0.064
[2,   900] loss: 0.048
[2,  1000] loss: 0.060
[2,  1100] loss: 0.064
[2,  1200] loss: 0.060
[2,  1300] loss: 0.071
[2,  1400] loss: 0.062
[2,  1500] loss: 0.074
[2,  1600] loss: 0.052
[2,  1700] loss: 0.053
[2,  1800] loss: 0.065
[2,  1875] Last loss: 0.050
Finished Training

Training Time: 00:00:26
Training Time Duration:  26.940658330

In [29]:
epoch_list = [x + 1 for x in list(epochs)]

table_data = {'epoch': epoch_list, 'loss_NMIST': loss_list}
df_training = pd.DataFrame(table_data)
df_training.set_index('epoch', inplace=True)
df_training

,loss_NMIST
epoch,
1,0.081255
2,0.049743


In [24]:
# Manually create the dataset_list
dataset_list = ["NMIST"]
#dataset_list = ["NMIST","CIFAR10","CIFAR100","FashionMNIST","EMNIST","KMNIST","QMNIST","STL10","USPS"] 

# Create a dictionary with the list values
data = {'Dataset': dataset_list,
        'Training Time (s)': training_time_list,
        'Testing Time (s)': testing_time_list,
        'Accuracy Test': accuracy_test_list,
        'Loss Test': loss_test_list}

# Create a DataFrame from the dictionary
df_model_dataset = pd.DataFrame(data)

df_model_dataset.set_index('Dataset', inplace=True)

# Display the DataFrame
df_model_dataset.transpose()

Dataset,NMIST
Training Time (s),27.287401
Testing Time (s),2.291809
Accuracy Test,0.988300
Loss Test,0.011700
